In [ ]:
import pandas as pd
import numpy as np

# 원-핫 인코더
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold

train_df_origin = pd.read_csv('train.csv')
train_df_extra = pd.read_csv('training_extra.csv')
train_df = pd.concat([train_df_origin, train_df_extra], axis=0)
train_df = train_df.drop('id', axis = 1)
train_df = train_df.reset_index().drop('index', axis = 1)
test_df = pd.read_csv('test.csv').drop('id', axis = 1)

<div style="background-color:gray; padding:10px; border-radius:10px; font-size:18px;">

데이터 전처리로 머신러닝 모델 유형에 따라 사용할 2가지 데이터 프레임 생성<br><br>
범주형 변수 결측치 처리 : 최빈값 대체<br>
연속형 변수 결측치 처리 : 중앙값 대체<br>
범주형 변수 인코딩 방법 : One-Hot 인코딩<br><br>

설명<br>
EDA에서 범주형 피쳐들을 순서인코딩할만한 경우를 찾지 못했기 때문에 원-핫인코딩 사용<br>
결측치를 대체하는 방법 역시 EDA에서 특별한 경우가 없으므로 일반적인 대체법을 사용<br>
</div>

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 1<br><br>
범주형 결측치 최빈값 대체<br>
연속형 결측치 중앙값 대체<br>
범주형 변수 원핫 인코딩<br>
</div>

In [2]:

# ------------------------------------------------- train 데이터 결측치 대체 ------------------------------------
# train 데이터 전처리
ce_train_df = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
ce_train_df['Missing_count'] = ce_train_df.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
Weight_median = ce_train_df['Weight Capacity (kg)'].median()
ce_train_df['Weight Capacity (kg)'] = ce_train_df['Weight Capacity (kg)'].fillna(Weight_median)

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_train_df.select_dtypes('object').columns)
for col in cat_cols:
    ce_train_df[col] = ce_train_df[col].fillna(ce_train_df[col].mode().iloc[0])

# ------------------------------------------------- test 데이터 결측치 대체 ------------------------------------
# test 데이터 전처리
ce_test_df = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
ce_test_df['Missing_count'] = ce_test_df.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
ce_test_df['Weight Capacity (kg)'] = ce_test_df['Weight Capacity (kg)'].fillna(Weight_median)
# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_test_df.select_dtypes('object').columns)
for col in cat_cols:
    ce_test_df[col] = ce_test_df[col].fillna(ce_train_df[col].mode().iloc[0])

# ------------------------------------------------- train 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_train_df.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_train_df[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_train_df, encoded_df], axis = 1)
ce_train_df = final_df

# ------------------------------------------------- test 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_test_df.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_test_df[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_test_df, encoded_df], axis = 1)
ce_test_df = final_df

# 인코딩 이전 변수 제거
ce_train_df = ce_train_df.drop(columns = cat_cols)
ce_test_df = ce_test_df.drop(columns = cat_cols)

print('train 데이터 크기', ce_train_df.shape)
print('test 데이터 크기', ce_test_df.shape)

train 데이터 크기 (3994318, 29)
test 데이터 크기 (200000, 28)


In [3]:

# ------------------------------------------ train 데이터 결측치 대체 --------------------------------------------------

c_train_df = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
c_train_df['Missing_count'] = c_train_df.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
c_train_df['Weight Capacity (kg)'] = c_train_df['Weight Capacity (kg)'].fillna(c_train_df['Weight Capacity (kg)'].median())

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(c_train_df.select_dtypes('object').columns)

for col in cat_cols:
    c_train_df[col] = c_train_df[col].fillna(c_train_df[col].mode().iloc[0])

# ------------------------------------------ test 데이터 결측치 대체 --------------------------------------------------
c_test_df = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
c_test_df['Missing_count'] = c_test_df.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
c_test_df['Weight Capacity (kg)'] = c_test_df['Weight Capacity (kg)'].fillna(c_train_df['Weight Capacity (kg)'].median())
# 범주형 변수 결측치 최빈값 대체
cat_cols = list(c_test_df.select_dtypes('object').columns)
for col in cat_cols:
    c_test_df[col] = c_test_df[col].fillna(c_train_df[col].mode().iloc[0])

In [4]:
# 타겟 분리 및 삭제
y_df = train_df['Price'].copy()
c_train_df = c_train_df.drop('Price', axis = 1).copy()
ce_train_df = ce_train_df.drop('Price', axis = 1).copy()

In [5]:
# 데이터 쓰기
y_df.to_csv('y_df', index = False)
c_train_df.to_csv('c_train_df', index = False)
ce_train_df.to_csv('ce_train_df',index = False)

In [6]:
c_test_df.to_csv('c_test_df', index = False)
ce_test_df.to_csv('ce_test_df', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 2<br><br>
타겟에 가장 영향을 주는 피처 Weight Capacity (kg)를 타겟인코딩
</div>

In [25]:
import numpy as np
import pandas as pd
from feature_engine.encoding import MeanEncoder
from sklearn.model_selection import KFold

# ✅ 1. 데이터 복사
ce_train_df2 = ce_train_df.copy()
ce_test_df2 = ce_test_df.copy()
ce_train_df2['Price'] = train_df['Price'].copy()

# TargetEncoder 객체 생성
encoder = TargetEncoder(cols=['Weight Capacity (kg)'], smoothing=20, min_samples_leaf=2)

# fit_transform을 사용하여 'Weight Capacity (kg)'를 타겟 인코딩
ce_train_df2['Weight Capacity TE'] = encoder.fit_transform(ce_train_df2['Weight Capacity (kg)'], ce_train_df2['Price'])

# test 데이터도 변환
test_sub = encoder.transform(ce_test_df['Weight Capacity (kg)'])

In [ ]:
# 제출
sub['Price'] = test_sub['Weight Capacity (kg)']
sub.to_csv('sub_2.csv', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 3<br><br>
타겟인코딩한 피처를 추가하여 학습<br>
범주형 결측치를 'Missing' 텍스트로 처리
</div>

In [20]:

# ------------------------------------------------- train 데이터 결측치 대체 ------------------------------------
# train 데이터 전처리
ce_train_df3 = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
ce_train_df3['Missing_count'] = ce_train_df3.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
Weight_median = ce_train_df3['Weight Capacity (kg)'].median()
ce_train_df3['Weight Capacity (kg)'] = ce_train_df3['Weight Capacity (kg)'].fillna(Weight_median)

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_train_df3.select_dtypes('object').columns)
for col in cat_cols:
    ce_train_df3[col] = ce_train_df3[col].fillna('Missing_value')

# ------------------------------------------------- test 데이터 결측치 대체 ------------------------------------
# test 데이터 전처리
ce_test_df3 = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
ce_test_df3['Missing_count'] = ce_test_df3.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
ce_test_df3['Weight Capacity (kg)'] = ce_test_df3['Weight Capacity (kg)'].fillna(Weight_median)
# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_test_df3.select_dtypes('object').columns)
for col in cat_cols:
    ce_test_df3[col] = ce_test_df3[col].fillna('Missing_value')

# ------------------------------------------------- train 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_train_df3.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_train_df3[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_train_df3, encoded_df], axis = 1)
ce_train_df3 = final_df

# ------------------------------------------------- test 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_test_df3.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_test_df3[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_test_df3, encoded_df], axis = 1)
ce_test_df3 = final_df

# 인코딩 이전 변수 제거
ce_train_df3 = ce_train_df3.drop(columns = cat_cols)
ce_test_df3 = ce_test_df3.drop(columns = cat_cols)

print('train 데이터 크기', ce_train_df3.shape)
print('test 데이터 크기', ce_test_df3.shape)

# --------------------------------------------------- train 타겟 인코딩 변수 추가 -----------------------------------

# TargetEncoder 객체 생성
encoder = TargetEncoder(cols=['Weight Capacity (kg)'], smoothing=20, min_samples_leaf=2)

# fit_transform을 사용하여 'Weight Capacity (kg)'를 타겟 인코딩
ce_train_df3['Weight Capacity TE'] = encoder.fit_transform(ce_train_df3['Weight Capacity (kg)'], ce_train_df3['Price'])

# -------------------------------------------------- test 타겟 인코딩 변수 추가

# test 데이터에도 타겟인코딩 변수 추가
test_sub = encoder.transform(ce_test_df3['Weight Capacity (kg)'])
ce_test_df3['Weight Capacity TE'] = test_sub

# 타겟 변수 삭제
ce_train_df3 = ce_train_df3.drop('Price', axis = 1)

print('train 데이터 크기', ce_train_df3.shape)
print('test 데이터 크기', ce_test_df3.shape)

train 데이터 크기 (3994318, 36)
test 데이터 크기 (200000, 35)
train 데이터 크기 (3994318, 36)
test 데이터 크기 (200000, 36)


In [23]:
# 학습 파일 생성
ce_train_df3.to_csv('ce_train_df3', index = False)
ce_test_df3.to_csv('ce_test_df3', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 5<br><br>
높은 중요도를 가지는 피처만 가진 데이터 프레임
</div>

In [2]:

# ------------------------------------------------- train 데이터 결측치 대체 ------------------------------------
# train 데이터 전처리
ce_train_df3 = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
ce_train_df3['Missing_count'] = ce_train_df3.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
Weight_median = ce_train_df3['Weight Capacity (kg)'].median()
ce_train_df3['Weight Capacity (kg)'] = ce_train_df3['Weight Capacity (kg)'].fillna(Weight_median)

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_train_df3.select_dtypes('object').columns)
for col in cat_cols:
    ce_train_df3[col] = ce_train_df3[col].fillna('Missing_value')

# ------------------------------------------------- test 데이터 결측치 대체 ------------------------------------
# test 데이터 전처리
ce_test_df3 = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
ce_test_df3['Missing_count'] = ce_test_df3.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
ce_test_df3['Weight Capacity (kg)'] = ce_test_df3['Weight Capacity (kg)'].fillna(Weight_median)
# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_test_df3.select_dtypes('object').columns)
for col in cat_cols:
    ce_test_df3[col] = ce_test_df3[col].fillna('Missing_value')

# ------------------------------------------------- train 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_train_df3.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_train_df3[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_train_df3, encoded_df], axis = 1)
ce_train_df3 = final_df

# ------------------------------------------------- test 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_test_df3.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_test_df3[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_test_df3, encoded_df], axis = 1)
ce_test_df3 = final_df

# 인코딩 이전 변수 제거
ce_train_df3 = ce_train_df3.drop(columns = cat_cols)
ce_test_df3 = ce_test_df3.drop(columns = cat_cols)

print('train 데이터 크기', ce_train_df3.shape)
print('test 데이터 크기', ce_test_df3.shape)

# --------------------------------------------------- train 타겟 인코딩 변수 추가 -----------------------------------

# TargetEncoder 객체 생성
encoder = TargetEncoder(cols=['Weight Capacity (kg)'], smoothing=20, min_samples_leaf=2)

# fit_transform을 사용하여 'Weight Capacity (kg)'를 타겟 인코딩
ce_train_df3['Weight Capacity TE'] = encoder.fit_transform(ce_train_df3['Weight Capacity (kg)'], ce_train_df3['Price'])

# -------------------------------------------------- test 타겟 인코딩 변수 추가

# test 데이터에도 타겟인코딩 변수 추가
test_sub = encoder.transform(ce_test_df3['Weight Capacity (kg)'])
ce_test_df3['Weight Capacity TE'] = test_sub

# 타겟 변수 삭제
ce_train_df3 = ce_train_df3.drop('Price', axis = 1)

ce_train_df4 = ce_train_df3[['Weight Capacity (kg)', 'Weight Capacity TE', 'Compartments', 'Missing_count']]
ce_test_df4 = ce_test_df3[['Weight Capacity (kg)', 'Weight Capacity TE', 'Compartments', 'Missing_count']]

print('train 데이터 크기', ce_train_df4.shape)
print('test 데이터 크기', ce_test_df4.shape)

train 데이터 크기 (3994318, 36)
test 데이터 크기 (200000, 35)
train 데이터 크기 (3994318, 4)
test 데이터 크기 (200000, 4)


In [3]:
# 학습 파일 생성
ce_train_df4.to_csv('ce_train_df4', index = False)
ce_test_df4.to_csv('ce_test_df4', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 6<br><br>
데이터 세트 1에서
결측치 최빈값처리 -> 결측치 Missing 처리
</div>

In [15]:
# ------------------------------------------------- train 데이터 결측치 대체 ------------------------------------
# train 데이터 전처리
ce_train_df5 = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
ce_train_df5['Missing_count'] = ce_train_df5.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
Weight_median = ce_train_df5['Weight Capacity (kg)'].median()
ce_train_df5['Weight Capacity (kg)'] = ce_train_df5['Weight Capacity (kg)'].fillna(Weight_median)

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_train_df5.select_dtypes('object').columns)
for col in cat_cols:
    ce_train_df5[col] = ce_train_df5[col].fillna('Missing')

# ------------------------------------------------- test 데이터 결측치 대체 ------------------------------------
# test 데이터 전처리
ce_test_df5 = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
ce_test_df5['Missing_count'] = ce_test_df5.isnull().sum(axis = 1)

# 연속형 변수 결측치 중앙값 대체
ce_test_df5['Weight Capacity (kg)'] = ce_test_df5['Weight Capacity (kg)'].fillna(Weight_median)

# 범주형 변수 결측치 최빈값 대체
cat_cols = list(ce_test_df5.select_dtypes('object').columns)
for col in cat_cols:
    ce_test_df5[col] = ce_test_df5[col].fillna('Missing')

# ------------------------------------------------- train 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_train_df5.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_train_df5[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_train_df5, encoded_df], axis = 1)
ce_train_df5 = final_df

# ------------------------------------------------- test 데이터 인코딩 ------------------------------------

# 원-핫 인코딩
cat_cols = list(ce_test_df5.select_dtypes('object').columns)
encoder = OneHotEncoder(sparse_output = False)
encoded_array = encoder.fit_transform(ce_test_df5[cat_cols])
encoded_columns = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_array, columns = encoded_columns)
final_df = pd.concat([ce_test_df5, encoded_df], axis = 1)
ce_test_df5 = final_df

# 인코딩 이전 변수 제거
ce_train_df5 = ce_train_df5.drop(columns = cat_cols)
ce_test_df5 = ce_test_df5.drop(columns = cat_cols)

ce_train_df5 = ce_train_df5.drop('Price', axis = 1).copy()

print('train 데이터 크기', ce_train_df5.shape)
print('test 데이터 크기', ce_test_df5.shape)

train 데이터 크기 (3994318, 35)
test 데이터 크기 (200000, 35)


In [19]:
ce_train_df5.to_csv('ce_train_df5', index = False)
ce_test_df5.to_csv('ce_test_df5', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 7<br><br>
cat_boost 전용 데이터 세트<br>
결측치만 Missing 처리함
</div>

In [2]:
# ------------------------------------------------- train 데이터 결측치 대체 ------------------------------------
# train 데이터 전처리
ce_train_df6 = train_df.copy()

# 결측치 개수를 기록한 피처 추가 생성
ce_train_df6['Missing_count'] = ce_train_df6.isnull().sum(axis = 1)


# 범주형 변수 결측치 missing 처리
cat_cols = list(ce_train_df6.select_dtypes('object').columns)
for col in cat_cols:
    ce_train_df6[col] = ce_train_df6[col].fillna('Missing')

# ------------------------------------------------- test 데이터 결측치 대체 ------------------------------------
# test 데이터 전처리
ce_test_df6 = test_df.copy()

# 결측치 개수를 기록한 피쳐 추가 생성
ce_test_df6['Missing_count'] = ce_test_df6.isnull().sum(axis = 1)

# 범주형 변수 결측치 missing 처리
cat_cols = list(ce_test_df6.select_dtypes('object').columns)
for col in cat_cols:
    ce_test_df6[col] = ce_test_df6[col].fillna('Missing')


# 타겟 분리 후 제거
ce_train_df6 = ce_train_df6.drop('Price', axis = 1).copy()

print('train 데이터 크기', ce_train_df6.shape)
print('test 데이터 크기', ce_test_df6.shape)

train 데이터 크기 (3994318, 10)
test 데이터 크기 (200000, 10)


In [7]:
ce_train_df6.to_csv('ce_train_df6', index = False)
ce_test_df6.to_csv('ce_test_df6', index = False)

<div style="background-color:red; padding:10px; border-radius:10px; font-size:18px;">
데이터 세트 8<br>
모든 변수 타겟인코딩한 데이터 세트<br>
</div>

In [39]:
ce_train_df8 = train_df.copy()
ce_test_df8 = test_df.copy()
y_df = ce_train_df8['Price']
ce_train_df8 = ce_train_df8.drop('Price', axis = 1)

# ------------------------------------------ train 결측치 개수 피처 생성 ------------------------------

train_missing_count = ce_train_df8.isnull().sum(axis = 1).copy()

# ------------------------------------------- train 결측치 처리 ---------------------------------------

Weight_capacity_median = ce_train_df8['Weight Capacity (kg)'].median()
Compartments_median = ce_train_df8['Compartments'].median()

ce_train_df8['Weight Capacity (kg)'] = ce_train_df8['Weight Capacity (kg)'].fillna(Weight_capacity_median)
ce_train_df8['Compartments'] = ce_train_df8['Compartments'].fillna(Compartments_median)

cat_col = list(ce_train_df8.select_dtypes('object').columns)

for col in cat_col:
    ce_train_df8[col] = ce_train_df8[col].fillna('Missing')

# ------------------------------------------ test 결측치 개수 피처 생성 ------------------------------

test_missing_count = ce_test_df8.isnull().sum(axis = 1).copy()

# ------------------------------------------- test 결측치 처리 ---------------------------------------

ce_test_df8['Weight Capacity (kg)'] = ce_test_df8['Weight Capacity (kg)'].fillna(Weight_capacity_median)
ce_test_df8['Compartments'] = ce_test_df8['Compartments'].fillna(Compartments_median)

cat_col = list(ce_test_df8.select_dtypes('object').columns)

for col in cat_col:
    ce_test_df8[col] = ce_test_df8[col].fillna('Missing')


In [17]:
# ------------------------------------------- train 타겟 인코더 생성 -----------------------------------------

# 변수 명 리스트
col_names = list(ce_train_df8.columns)

# 교차검증 객체 생성
kf = KFold(n_splits= 25, shuffle= True, random_state = 1)

# 인코딩된 데이터가 들어갈 데이터 프레임 준비
te_df = ce_train_df8.copy()

for col in col_names:

    # 변수명 
    te_df[col + '_TE'] = np.nan

    for train_index, val_index in kf.split(ce_train_df8):

        # fold 분할
        train_fold, val_fold = te_df.iloc[train_index], te_df.iloc[val_index]

        # smoothing 값이 크면 전체 평균을 더욱 반영, 작으면 카테고리별 평균을 더욱 반영
        # min_samples_leaf = n 일때, 특정 카테고리에 속하는 샘플 수가 n보다 작으면 전체평균을 더욱 반영
        # col을 중괄호로 감싸는 이유는 TargetEncoder는 원래 여러개의 변수를 리스트로 받도록 설계되었기 때문에,
        # 변수가 한개라도 리스트로 전달해야한다
        encoder = TargetEncoder(cols = [col], smoothing= 20, min_samples_leaf=2)

        # fit 함수로 카테고리별 평균을 계산 (계산만 해줌) 
        # (col을 중괄호로 두번 감싸는 이유는 fit에서는 데이터 프레임만 처리할 수 있기 때문이다(타겟은 시리즈여도 된다))
        encoder.fit(train_fold[[col]], y_df.iloc[train_fold.index])

        # transform 메서드로 검증데이터에 카테고리별 평균으로 변환
        te_df.loc[val_index, col + '_TE'] = encoder.transform(val_fold[[col]])
    
    encoder.fit(ce_train_df8[[col]], y_df)
    ce_test_df8[col + '_TE'] = encoder.transform(ce_test_df8[[col]])

In [ ]:
# 위의 코드로 하면 te 어쩌구 변수가 모두 Nan값으로 나와서 이걸로 했다


# ------------------------------------------- train 타겟 인코더 생성 -----------------------------------------

# 변수 명 리스트
col_names = list(ce_train_df8.columns)

# 교차검증 객체 생성
kf = KFold(n_splits= 25, shuffle= True, random_state = 1)

# 인코딩된 데이터가 들어갈 데이터 프레임 준비
te_df = ce_train_df8.copy()

for col in col_names:
    # 새로운 컬럼을 NaN 값으로 초기화
    te_df[col + '_TE'] = np.nan

    for train_index, val_index in kf.split(ce_train_df8):
        # fold 분할
        train_fold, val_fold = ce_train_df8.iloc[train_index], ce_train_df8.iloc[val_index]

        # TargetEncoder 적용
        encoder = TargetEncoder(cols=[col], smoothing=20, min_samples_leaf=2)

        # fit 함수 실행
        encoder.fit(train_fold[[col]], y_df.iloc[train_fold.index])

        # transform 결과를 따로 저장하여, 인덱스를 명확하게 지정
        transformed_vals = encoder.transform(val_fold[[col]])
        
        # 인덱스를 직접 매칭하여 할당
        te_df.loc[val_fold.index, col + '_TE'] = transformed_vals.values.flatten()

    # 전체 데이터로 최종 fit 후, 테스트 데이터 변환
    encoder.fit(ce_train_df8[[col]], y_df)
    ce_test_df8[col + '_TE'] = encoder.transform(ce_test_df8[[col]])

In [41]:
te_cols = ['Brand_TE','Material_TE', 'Size_TE', 'Compartments_TE',
    'Laptop Compartment_TE','Waterproof_TE', 'Style_TE', 'Color_TE',
    'Weight Capacity (kg)_TE']

te_train_df = te_df.copy()
te_test_df = ce_test_df8.copy()

# 인코딩 변수만 남기기
te_train_df = te_train_df[te_cols]
te_test_df = te_test_df[te_cols]

# 결측치 수를 기록한 피처 추가
te_train_df['Missing_count'] = train_missing_count
te_test_df['Missing_count'] = test_missing_count

In [45]:
te_train_df.to_csv('te_train_df', index = False)
te_test_df.to_csv('te_test_df', index = False)